# Mice sleep

## Data reading, normalization/pre-processing, and seperation

In [6]:
import pandas as pd
import numpy as np

In [15]:
# The file is made up of the following structure:
# state : char (1 byte)
# bin : array[0..400] of single (array of 401 floats (4 bytes each))
# EEGv,EMGv,temp : single (4 bytes each)
# Make a pandas dataframe with the data with appropriate column names
# Create a dtype with the binary data format and the desired column names
file = './data/DBA10.smo'
dt = [('state', 'i1')]
for i in range(401):
    dt.append(('bin' + str(i), 'f4'))
dt.append(('EEGv', 'f4'))
dt.append(('EMGv', 'f4'))
dt.append(('temp', 'f4'))
dt = np.dtype(dt)
data = np.fromfile(file, dtype=dt)
# Or if you want to explicitly set the column names
df = pd.DataFrame.from_records(data)
df.head()
            

,state,bin0,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,...,bin394,bin395,bin396,bin397,bin398,bin399,bin400,EEGv,EMGv,temp
0,119,0.000154,0.000048,0.000003,2.148770e-05,1.632360e-05,1.604630e-05,1.569020e-05,9.569190e-06,2.992340e-05,...,4.778510e-10,3.952580e-10,3.429190e-09,6.106250e-09,3.552800e-09,3.909590e-09,1.493170e-08,4.719158e-09,1.597187e-09,0.0
1,119,0.000052,0.000084,0.000009,7.071300e-06,9.172860e-06,4.433370e-06,3.053690e-06,8.397030e-06,8.254330e-06,...,1.958490e-09,2.502730e-09,2.280000e-09,5.343420e-10,2.715390e-10,6.042040e-10,1.768590e-09,4.708915e-09,1.839138e-09,0.0
2,119,0.000004,0.000003,0.000009,2.246900e-06,8.937540e-06,3.990790e-06,8.024030e-07,4.551380e-06,6.409220e-06,...,1.049450e-10,4.993360e-10,1.033830e-09,2.020390e-10,9.781460e-10,6.192400e-09,1.743470e-08,4.812772e-09,2.612307e-09,0.0
3,119,0.000002,0.000015,0.000039,6.720840e-06,5.050530e-07,8.436160e-07,1.011240e-06,6.865460e-07,7.357910e-08,...,3.089290e-09,1.684690e-09,2.604550e-09,2.139050e-09,1.311040e-09,5.807190e-09,1.398960e-08,4.208398e-09,8.921635e-10,0.0
4,119,0.000129,0.000089,0.000011,7.559840e-07,2.077260e-06,4.817490e-06,2.384510e-05,2.080720e-05,2.435700e-05,...,1.017240e-09,1.610110e-10,8.243920e-10,3.875150e-10,1.901600e-09,4.381190e-09,1.166410e-08,4.206695e-09,4.336937e-09,0.0


## Feature Selection

In [ ]:
# Selecting features

## Model building

### Model 1: Linear Regression

### Model 2: Random Forest

### Model 3: Decision Tree

### Model 4: Perceptron

### Model 5: CNN

## Model Comparison